In [36]:
import pandas as pd
import json

In [4]:
from kafka import KafkaConsumer

kafka_broker = "localhost:9093"

In [49]:
consumer = KafkaConsumer(
    'abstract',
    bootstrap_servers=[kafka_broker],
    enable_auto_commit=True,
    value_deserializer=lambda x: x.decode('utf-8'),
    consumer_timeout_ms=60000
)

In [50]:
data = []
for message in consumer:
    data.append(json.loads(message.value))
consumer.close()

In [38]:
print(data)

[{'title': 'Shear capacity of high performance fiber reinforced concrete I-beams', 'id': 'SCOPUS_ID:85029781991', 'subject-area': ['Civil and Structural Engineering', 'Building and Construction', 'Materials Science (all)'], 'year': '2017', 'citation_count': '55', 'authors': [{'@_fa': 'true', '@auid': '35743586000', '@seq': '1', 'affiliation': {'@href': 'https://api.elsevier.com/content/affiliation/affiliation_id/60028190', '@id': '60028190'}, 'author-url': 'https://api.elsevier.com/content/author/author_id/35743586000', 'ce:given-name': 'Withit', 'ce:indexed-name': 'Pansuk W.', 'ce:initials': 'W.', 'ce:surname': 'Pansuk', 'preferred-name': {'ce:given-name': 'Withit', 'ce:indexed-name': 'Pansuk W.', 'ce:initials': 'W.', 'ce:surname': 'Pansuk'}}, {'@_fa': 'true', '@auid': '57192267714', '@seq': '2', 'affiliation': {'@href': 'https://api.elsevier.com/content/affiliation/affiliation_id/60028190', '@id': '60028190'}, 'author-url': 'https://api.elsevier.com/content/author/author_id/571922677

In [16]:
from io import StringIO

In [39]:
from itertools import combinations

In [40]:
df = pd.DataFrame.from_dict(data)

In [41]:
display(df)

,title,id,subject-area,year,citation_count,authors,affiliations
0,Shear capacity of high performance fiber reinf...,SCOPUS_ID:85029781991,"[Civil and Structural Engineering, Building an...",2017,55,"[{'@_fa': 'true', '@auid': '35743586000', '@se...",[{'@href': 'https://api.elsevier.com/content/a...
1,Mutation operators in BPMN model,SCOPUS_ID:85049840335,"[Software, Human-Computer Interaction, Compute...",2017,0,"[{'@_fa': 'true', '@auid': '57202922236', '@se...",[{'@href': 'https://api.elsevier.com/content/a...
2,Phoyunnanin E inhibits migration of non-small ...,SCOPUS_ID:85039703338,[Complementary and Alternative Medicine],2017,29,"[{'@_fa': 'true', '@auid': '56560089400', '@se...",[{'@href': 'https://api.elsevier.com/content/a...
3,Heat transfer behavior and physiological effec...,SCOPUS_ID:85039924885,[Engineering (all)],2017,1,"[{'@_fa': 'true', '@auid': '37061132300', '@se...",[{'@href': 'https://api.elsevier.com/content/a...
4,Investigation of alkoxysilanes in the presence...,SCOPUS_ID:85039924338,[Engineering (all)],2017,4,"[{'@_fa': 'true', '@auid': '57191345705', '@se...",[{'@href': 'https://api.elsevier.com/content/a...
5,Input modeling prioritization using statistica...,SCOPUS_ID:85039922690,[Engineering (all)],2017,0,"[{'@_fa': 'true', '@auid': '57190255581', '@se...",[{'@href': 'https://api.elsevier.com/content/a...
6,Analytic hierarchy process for stakeholder par...,SCOPUS_ID:85039922521,[Engineering (all)],2017,8,"[{'@_fa': 'true', '@auid': '57200139530', '@se...",[{'@href': 'https://api.elsevier.com/content/a...
7,Parametric study of spot welding between li-io...,SCOPUS_ID:85039922510,[Engineering (all)],2017,10,"[{'@_fa': 'true', '@auid': '57200138935', '@se...",[{'@href': 'https://api.elsevier.com/content/a...
8,A new wavelet denoising method for noise thres...,SCOPUS_ID:85039921418,[Engineering (all)],2017,22,"[{'@_fa': 'true', '@auid': '57217958543', '@se...",[{'@href': 'https://api.elsevier.com/content/a...


In [43]:
df.head(1)

,title,id,subject-area,year,citation_count,authors,affiliations
0,Shear capacity of high performance fiber reinf...,SCOPUS_ID:85029781991,"[Civil and Structural Engineering, Building an...",2017,55,"[{'@_fa': 'true', '@auid': '35743586000', '@se...",[{'@href': 'https://api.elsevier.com/content/a...


In [42]:
df.authors[0]

[{'@_fa': 'true',
  '@auid': '35743586000',
  '@seq': '1',
  'affiliation': {'@href': 'https://api.elsevier.com/content/affiliation/affiliation_id/60028190',
   '@id': '60028190'},
  'author-url': 'https://api.elsevier.com/content/author/author_id/35743586000',
  'ce:given-name': 'Withit',
  'ce:indexed-name': 'Pansuk W.',
  'ce:initials': 'W.',
  'ce:surname': 'Pansuk',
  'preferred-name': {'ce:given-name': 'Withit',
   'ce:indexed-name': 'Pansuk W.',
   'ce:initials': 'W.',
   'ce:surname': 'Pansuk'}},
 {'@_fa': 'true',
  '@auid': '57192267714',
  '@seq': '2',
  'affiliation': {'@href': 'https://api.elsevier.com/content/affiliation/affiliation_id/60028190',
   '@id': '60028190'},
  'author-url': 'https://api.elsevier.com/content/author/author_id/57192267714',
  'ce:given-name': 'Thuc N.',
  'ce:indexed-name': 'Nguyen T.N.',
  'ce:initials': 'T.N.',
  'ce:surname': 'Nguyen',
  'preferred-name': {'ce:given-name': 'Thuc N.',
   'ce:indexed-name': 'Nguyen T.N.',
   'ce:initials': 'T.N.',

In [44]:
df.affiliations[0]

[{'@href': 'https://api.elsevier.com/content/affiliation/affiliation_id/60118019',
  '@id': '60118019',
  'affilname': 'Faculteit Civiele Techniek en Geowetenschappen, TU Delft'},
 {'@href': 'https://api.elsevier.com/content/affiliation/affiliation_id/60028190',
  '@id': '60028190',
  'affilname': 'Chulalongkorn University'},
 {'@href': 'https://api.elsevier.com/content/affiliation/affiliation_id/60014652',
  '@id': '60014652',
  'affilname': 'Hokkaido University'}]

In [26]:
# Prepare nodes from affiliations
affiliations_dict = {}
for aff_list in df.affiliations.dropna():
    for aff in aff_list:
        aff_id = aff['@id']
        if aff_id not in affiliations_dict:
            affiliations_dict[aff_id] = aff['affilname']

nodes = pd.DataFrame(list(affiliations_dict.items()), columns=['Id', 'Label'])
nodes.to_csv('nodes-affiliation.csv', index=False)

# Prepare edges
edges_list = []

# Function to safely get the affiliation ID directly from the author dictionary
def get_affiliation_id(author):
    try:
        return author['affiliation']['@id']
    except (KeyError, TypeError):
        return None

# Iterate through each paper
for index, row in df.iterrows():
    # Check if authors exist and the list is not empty
    if row['authors'] and isinstance(row['authors'], list):
        paper_affiliations = [get_affiliation_id(author) for author in row['authors'] if get_affiliation_id(author) is not None]

        # Create edges between every pair of different affiliations
        unique_affiliations = set(paper_affiliations)
        for aff1 in unique_affiliations:
            for aff2 in unique_affiliations:
                if aff1 != aff2:
                    # Check if this edge is already added
                    existing_edge = next((item for item in edges_list if (item['Source'] == aff1 and item['Target'] == aff2) or (item['Target'] == aff1 and item['Source'] == aff2)), None)
                    if existing_edge:
                        existing_edge['Weight'] += 1
                    else:
                        edges_list.append({'Source': aff1, 'Target': aff2, 'Weight': 1})

edges = pd.DataFrame(edges_list)
edges.to_csv('edges-affiliation.csv', index=False)


In [23]:
# Prepare nodes: Unique Subject Areas
subject_areas = set()
df['subject-area'].dropna().apply(lambda x: subject_areas.update(x))
nodes = pd.DataFrame(list(subject_areas), columns=['Label'])
nodes['Id'] = range(len(nodes))

# Prepare edges
edges_list = []

# Mapping from subject area to node Id
area_to_id = {area: idx for idx, area in enumerate(subject_areas)}

# Generate edges from papers' subject areas
for areas in df['subject-area'].dropna():
    if len(areas) > 1:
        for (area1, area2) in combinations(areas, 2):
            # Sort to avoid duplicate edges (area1, area2) and (area2, area1)
            area1, area2 = sorted([area1, area2])
            edge_key = (area_to_id[area1], area_to_id[area2])
            # Find edge if it already exists
            found_edge = next((e for e in edges_list if e['Source'] == edge_key[0] and e['Target'] == edge_key[1]), None)
            if found_edge:
                found_edge['Weight'] += 1
            else:
                edges_list.append({'Source': edge_key[0], 'Target': edge_key[1], 'Weight': 1})

edges = pd.DataFrame(edges_list)

# Export CSV files
nodes.to_csv('nodes-subject-area.csv', index=False)
edges.to_csv('edges-subject-area.csv', index=False)


In [25]:
# Prepare nodes: Unique authors
authors_dict = {}
for index, row in df.iterrows():
    for author in row['authors']:
        auid = author['@auid']
        name = f"{author['ce:given-name']} {author['ce:surname']}"
        authors_dict[auid] = name

nodes = pd.DataFrame(list(authors_dict.items()), columns=['Id', 'Label'])
nodes.to_csv('nodes-author-collaboration.csv', index=False)

# Prepare edges
edges_list = []

# Generate edges based on co-authorships
for index, row in df.iterrows():
    if len(row['authors']) > 1:
        # Generate all combinations of authors for the paper
        for (author1, author2) in combinations(row['authors'], 2):
            auid1, auid2 = author1['@auid'], author2['@auid']
            # Sort to avoid duplicate edges (auid1, auid2) and (auid2, auid1)
            auid1, auid2 = sorted([auid1, auid2])
            edge_key = (auid1, auid2)
            # Find edge if it already exists
            found_edge = next((e for e in edges_list if e['Source'] == edge_key[0] and e['Target'] == edge_key[1]), None)
            if found_edge:
                found_edge['Weight'] += 1
            else:
                edges_list.append({'Source': edge_key[0], 'Target': edge_key[1], 'Weight': 1})

edges = pd.DataFrame(edges_list)
edges.to_csv('edges-author-collaboration.csv', index=False)
